# DDF lensed transient single exposures access

This repo shows how to access the catalog of lensed transients inserted in the DC2's deep drilling fields and how to pull single callibrated exposure images and light curves. So far only the first two year worth of simulation have been processed. If you find that there are more recent processings available that are not being used here, please feel free to submit an issue [here](https://github.com/LSST-strong-lensing/DC2-notebooks/issues).

This notebook is largely based on [DP0 notebook tutorials](https://github.com/rubin-dp0/tutorial-notebooks), initially written by Melissa Graham. 

**Created:** 05-11-2022 by Rémy Joseph \
**Last reviewed:** 06-09-2022 by Rémy Joseph \
**Kernel used:** `desc-stack-weekly`

## Introduction

Lensed transients and their host and lensed galaxies were introduced in DC2 run 3.1i as part of the [Lens Sprinkler project](https://portal.lsstdesc.org/DESCPub/app/PB/show_project?pid=35) led by Bryce Klambach and Ji Won Park. Their work should be acknowledged in any published materials that makes use of this work.
The Deep Drilling Fields (DDF) were injected with a sample of a few 1000s lensed quasars and supernovae. This notebook is intended to help anyone who might want to use this sample to run their own finding or analysis tools.

This notebook makes use of the gen3 butler developped by the project. More information can be found in the [LSST science pipelines documentation](https://pipelines.lsst.io/getting-started/dc2-guide.html). 

In [ ]:
# Imports
# Generic python packages
import numpy as np
import pylab as plt
from tqdm.notebook import tqdm
import pandas as pd
import sqlite3
import sqlalchemy 
from astropy.coordinates import SkyCoord
import astropy.units as u
import astropy
from astropy.time import Time

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
# Some plotting tools
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.transforms import Affine2D
import matplotlib.cm as cm

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.daf.base as dafBase
import lsst.afw.display as afwDisplay
from astropy.visualization import make_lupton_rgb
from lsst.afw.image import MultibandExposure
import lsst.geom as geom
import lsst.sphgeom as sphgeom
import lsst.afw.coord as afwCoord
import lsst.geom as afwGeom
afwDisplay.setDefaultBackend('matplotlib')

# imports python's garbage collector
import gc                            

import warnings
warnings.simplefilter("ignore", category=UserWarning)

import mpl_toolkits.axisartist.floating_axes as floating_axes
from mpl_toolkits.axisartist.grid_finder import DictFormatter, MaxNLocator

The Butler is the object that enables us to retrieve, read and write data. To create the Butler, we need to provide it with a path to the data set, which is called a "data repository". Butler repositories have both a database component and a file-like storage component; the latter can can be remote (i.e., pointing to an S3 bucket) or local (i.e., pointing to a directory on the local file system), and it contains a configuration file (usually butler.yaml) that points to the right database.

At the moment only the first 2 years-worth of observation of the DDF have calexp images available. 

In [ ]:
# This is the repo containing the Run3.1i DDF data:
repo = '/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo'

# These are the collections containing the Y1 processed visit images, etc..
collections = ['u/descdm/sfp_ddf_visits_part_00',
               'u/descdm/sfp_ddf_visits_part_01',
              ]

# Create a data butler and registry
butler = dafButler.Butler(repo, collections=collections)
registry = butler.registry


In [ ]:
def is_in_time(ra, 
               dec, 
               butler=None, 
               time=None, 
               skymap=None, 
               regions=[{"time": [[Time('2022-01-01T03:21:12.552', 
                                         format='isot', 
                                         scale='utc'),
                                   Time('2023-12-31T06:55:22.651', 
                                         format='isot', 
                                         scale='utc')]]}]):
    """
    Returns True if a set of coordinates (including time) is in a given region.
    
    Parameters
    ----------
    ra, dec: float
        Ra and Dec Coordinates.
    butler: lsst.daf.persistence.Butler
        servant providing access to a data repository
    time: astropy.time
        time of observation (event).
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    regions: array
        Array of dictionnaries containing available patches, tracts and time limits.
        
    Returns
    -------
    is_in: bool
        True if ‘(ra, dec)‘ is in ‘region‘
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    
    for r in regions:
        if time is not None:
            if np.size(r["time"]) > 0:
                for span in r["time"]:
                    if (time < span[1]) and (time > span[0]):
                        return True
        else:
            return True

    return False

# Load the lensed transient truth

Here we load the catalogs for inserted lensed transients that were generated as part of the LensSprinkler project. These are the truth tables that give the positions of the simulated lensed transients as they were inserted. 

It is possible to overlay the patches that have been processed so far. Note that it takes a few seconds to build so by default that display is not produced, but if you set ‘show_patches‘ ti ‘True‘, the patches will appear on the plot.

In [ ]:
# Databases and location for truth catalogs
folder = '/global/cfs/cdirs/descssim/DC2/Run3.0i/truth_tables/'
truth_sn = 'updated_lensed_sne_truth.db'
truth_agn = 'updated_lensed_agn_truth.db'

# Set to true to visualize the tract:4848, patches:35,36,42,43 area.
show_patches = True

try:
    conn_sn = sqlite3.connect(folder+truth_sn)   
    conn_agn = sqlite3.connect(folder+truth_agn)   
except Exception as e:
    print(e)

#Now in order to read in pandas dataframe we need to know table name
cursor = conn_sn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

sn = pd.read_sql_query('SELECT * FROM lensed_sne', conn_sn)
conn_sn.close()

cursor = conn_agn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

agn = pd.read_sql_query('SELECT * FROM lensed_agn', conn_agn)
conn_agn.close()

fig, ax = plt.subplots()
plt.title('Positions of lensed transients', fontsize = 30)
plt.plot(sn.ra, sn.dec, 'or', label = 'SN')
plt.plot(agn.ra, agn.dec, 'ob', label = 'AGN')
plt.xlabel('Ra', fontsize = 20)
plt.ylabel('Dec', fontsize = 20)
plt.legend(fontsize = 15, loc="upper right")


plt.show()

In [ ]:
def get_lens_radec(cat, 
                   butler=None, 
                   regions=[{"time": [[Time('2022-01-01T03:21:12.552', 
                                           format='isot', 
                                           scale='utc'),
                                      Time('2023-12-31T06:55:22.651', 
                                           format='isot', 
                                           scale='utc')]]}], 
                   time=False, 
                   skymap=None):
    
    if skymap is None and butler is not None:
        skymap = butler.get("skymap")
    inumber = -1
    radec = []
    lens_radec = []
    for index, row in cat.iterrows():
        if row.image_number == inumber+1:
            radec.append([row.ra, row.dec])
        else: 
            #assert len(radec) in [2,4], f"there should be either 2 or 3 images in a lens system but {len(radec)} were found"
            lens_ra, lens_dec = np.mean(radec, axis=0)
            # Only saves the lenses that are in the processed regions
            if time == True:
                t = Time(row.t0+row.t_delay, format='mjd')
            else:
                t = None
            if regions is not None:
                if is_in_time(lens_ra, lens_dec, butler, time=t, skymap=skymap, regions = regions):
                    lens_radec.append({'coord': np.mean(radec, axis=0), 'image_pos':radec, 't0':t, 'n_images':inumber+1})
            else:
                lens_radec.append({'coord': np.mean(radec, axis=0), 'image_pos':radec, 't0':t, 'n_images':inumber+1})
            radec = []
            radec.append([row.ra, row.dec])
            
        inumber = row.image_number
    return lens_radec

agn_lenses = get_lens_radec(agn)
sn_lenses = get_lens_radec(sn, time=True)

print(f"There are {len(agn_lenses)} AGNs in the DC2 3.1i DDF and {len(sn_lenses)} supernovae out of {len(get_lens_radec(sn))} are exploding during Y1-Y2.")


To view all the information available about each image, in each catalog, uncomment the following line:

In [ ]:
#print(sn.columns)
#print(agn.columns)

This catalog contains the positions for each image of a strongly lensed transient. This means that a quadrupely lensed supernova will have 4 entries in the catalog: one for each image. In the next box we produce the approximate coordinates of the lenses by averaging over the image coordinates. 

## Querying for calexp data

The database side of a data repository is called a registry. The registry contains entries for all data products, and organizes them by collection, dataset type, and data ID. Use the registry to investigate a repository by listing all collections. Specifying collections allow to browse nested collections.

Arbitrary spatial queries are not supported at this time, such as the "POINT() IN (REGION)" example found in this Butler queries documentation. In other words, at this time it is only possible to do queries involving regions that are already "in" the data repository, either because they are HTM pixel regions or because they are tract/patch/visit/visit+detector regions.

Thus, for this example we use the set of dimensions that correspond to different levels of the HTM (hierarchical triangular mesh) pixelization of the sky (HTM primer). The process is to transform a region or point into one or more HTM identifiers (HTM IDs), and then create a query using the HTM ID as the spatial data ID. The lsst.sphgeom library supports region objects and HTM pixelization in the LSST Science Pipelines.

The following function is meant to query for data sets that include a desired coordinate and observation date.

We need to provide precise spatial and temporal information stored in the registry, which are represented in Python by Timespan and Region objects, respectively. DimensionRecord objects that represent spatial or temporal concepts (a visit is both) have these objects attached to them.

In [ ]:

def query_transient(registry, radec, timespan, myband = 'r', mesh_lvl=10, time_limit=None):
    """ Query the registry for temporal and spatial traget.
    
    Parameters
    ----------
    radec: `SkyCoord` object
        the coordinates of the object to query

    timespan: `dafButler.timespan` object
        time span for the object.
        
    mesh_lvl(optional): int
        the level at which one sky pixel is about five arcmin across.
    
    """
    
    # Get the hierarchical triangular mesh (HTM) for the position to query. Initialize a sky pixelization.
    pixelization = sphgeom.HtmPixelization(mesh_lvl)
    # Find the HTM ID for a desired sky coordinate.
    htm_id = pixelization.index(sphgeom.UnitVector3d(sphgeom.LonLat.fromDegrees(
                            radec.ra.value, radec.dec.value)))
    
    # Query the datasets.
    datasetRefs = registry.queryDatasets("calexp", htm20=htm_id, 
                                     where=f"visit.timespan OVERLAPS my_timespan AND band = myband",
                                     bind={"my_timespan": timespan, "myband": myband})
    
    return datasetRefs

def remove_figure(fig):
    """Remove a figure to reduce memory footprint. """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()      # clear the figure
    plt.close(fig) # close the figure
    gc.collect()   # call the garbage collector

## Extracting patches for single exposures

Now we have functions that can query the datasets for specific times and locations. Knowing the time and place where Supernovae explode in our simulated set, we can therefore extract the patches that correspond to a SN exploding and sort the patches to display the evolution of a SN in time.

In [ ]:
def display_time_sorted_patches(catalog, registry, band='r', cutout_extent=20, limit=10):
    """ Display patches starting at the beginning of an explosion and sorts them according to their time stamp.
    
    Parameters
    ----------
    catalog: list
        catalog of lensed objects with reference location
    registry: butler.registry
        registry to query the dataset.
    band: string
        The optical band to display.
    cutout_extent: int
        The number of pixels-on-a-side to cut a patch out.
    limit: int
        Limits the number of images to display.
    
    """
    
    for row in catalog:
    
        # Get a timespan object for the temporal side of the query.
        if row['t0'] is not None:
            timespan = dafButler.Timespan(Time( row['t0'], format='mjd'), Time(row['t0']+10, format='mjd'))
        else:
            timespan = dafButler.Timespan(Time('2022-01-01T03:21:12.552', 
                                         format='isot', 
                                         scale='utc'),
                                   Time('2023-12-31T06:55:22.651', 
                                         format='isot', 
                                         scale='utc'))
        # Get the physical location for the query
        radec = SkyCoord(ra=row['coord'][0] * u.deg, dec= row['coord'][1] * u.deg)

        #Querying the registry for calexps at specific location and time interval (t0+10 days)
        dataset_i = query_transient(registry, radec, timespan, myband='i')
        dataset_g = query_transient(registry, radec, timespan, myband='g')
        dataset_r = query_transient(registry, radec, timespan, myband='r')

        # Let's visualize the dates at which observations were taken
        dates_g = [ref.dataId.timespan.begin.mjd for ref in dataset_g.expanded()]
        dates_r = [ref.dataId.timespan.begin.mjd for ref in dataset_r.expanded()]
        dates_i = [ref.dataId.timespan.begin.mjd for ref in dataset_i.expanded()]

        if len(list(dataset_r))>1:
            plt.title("MJD of observations")
            plt.hist(dates_g, label = f'g {len(dates_g)} observations', alpha=0.2)
            plt.hist(dates_r, label = f'r {len(dates_r)} observations', alpha=0.2)
            plt.hist(dates_i, label = f'i {len(dates_i)} observations', alpha=0.2)
            plt.legend()
            plt.show()
        
        # So far, we restrin display to i-band only.
        dates = dates_i
        args = np.argsort(dates)
        dates = np.array(dates)[args]

        mjd_arr = []
        calexp_arr = []
        flux_arr = []
        for i, ref in enumerate(tqdm(np.array(list(dataset_i))[args][:limit])):
            transient_pos = row['image_pos']

            # extract individual exposure with the butler.
            calexp = butler.get('calexp', dataId=ref.dataId)

            wcs = calexp.wcs

            # Get the pixel position for the lensed images
            point = geom.SpherePoint(row['coord'][0], row['coord'][1], geom.degrees)
            xy = geom.PointI(wcs.skyToPixel(point))

            if xy[0]<4072-cutout_extent and xy[1]<4000-cutout_extent and xy[0]>cutout_extent and xy[1]>cutout_extent:

                bbox = afwGeom.Box2I()
                bbox.include(afwGeom.Point2I(xy.x - cutout_extent,
                                             xy.y - cutout_extent))
                bbox.include(afwGeom.Point2I(xy.x + cutout_extent,
                                             xy.y + cutout_extent))
                x, y = [], []
                for pos in transient_pos:
                    pos = geom.SpherePoint(pos[0], pos[1], geom.degrees)

                    xp, yp = wcs.skyToPixel(pos)
                    x.append(xp-bbox.getMinX())
                    y.append(yp-bbox.getMinY())

                mat = np.eye(3)
                mat[:2,:2] = wcs.getCdMatrix()
                transform = Affine2D(mat)

                image = calexp[bbox]
                
                
                mjd_arr.append(dates[i])

                fig = plt.figure()
                arr = image.maskedImage.image.array#[xy.x-50:xy.x+50, xy.y-50:xy.y+50]
                flux_arr.append(np.sum(arr))

                plot_extents = 0, bbox.width, 0, bbox.height
                helper = floating_axes.GridHelperCurveLinear(
                    transform, plot_extents, 
                    tick_formatter1=DictFormatter({}),
                    tick_formatter2=DictFormatter({}),
                    grid_locator1=MaxNLocator(nbins=1),
                    grid_locator2=MaxNLocator(nbins=1),
                )
                ax = floating_axes.FloatingSubplot(fig, 111, grid_helper=helper)
                ax.imshow(arr, transform=transform+ax.transData, interpolation = 'nearest', cmap='gist_stern')
                ax.plot(x, y, 'ok', markersize=5, transform=transform+ax.transData)
                ax.set_title(f"Ra: {str(row['coord'][0])[:10]} Dec: {str(row['coord'][1])[:10]} MJD: {str(dates[i])[:10]} band: {ref.dataId['band']}")
                #ax.scatter(
                #    xy.x - bbox.minX, 
                #    xy.y - bbox.minY, 
                #    c='r', marker='+', transform=transform+ax.transData
                #)

                fig.add_subplot(ax)
                plt.show()

                # clean up memory
                remove_figure(fig)
        if len(flux_arr)>1:
            plt.title("light curve", fontsize=45)
            plt.plot(np.array(mjd_arr), np.array(flux_arr), 'o')
            plt.xlabel("MJD", fontsize=25)
            plt.ylabel("flux", fontsize=25)
            plt.show()
    pass

In [ ]:

display_time_sorted_patches(sn_lenses, registry, cutout_extent=20, limit = 40)


In [ ]:
display_time_sorted_patches(agn_lenses, registry, cutout_extent=20, limit = 40)